In [1]:
import pandas as pd
from pandas import DataFrame, Series
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
import random

import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

from datetime import date
from dateutil.relativedelta import relativedelta

In [2]:
TRIP_FILE = ('201508_trip_data.csv')
WEATHER = ('201508_weather_data.csv')
STATION_BALANCE = ('201508_status_data.csv')
STATION_DATA = ('201508_station_data.csv')

trip_data = pd.read_csv(TRIP_FILE)
weather_data = pd.read_csv(WEATHER)
station_balance_data = pd.read_csv(STATION_BALANCE)
station_data = pd.read_csv(STATION_DATA)

trip_data.head()

,Trip ID,Duration,Start Date,Start Station,Start Terminal,End Date,End Station,End Terminal,Bike #,Subscriber Type,Zip Code
0,913460,765,8/31/2015 23:26,Harry Bridges Plaza (Ferry Building),50,8/31/2015 23:39,San Francisco Caltrain (Townsend at 4th),70,288,Subscriber,2139
1,913459,1036,8/31/2015 23:11,San Antonio Shopping Center,31,8/31/2015 23:28,Mountain View City Hall,27,35,Subscriber,95032
2,913455,307,8/31/2015 23:13,Post at Kearny,47,8/31/2015 23:18,2nd at South Park,64,468,Subscriber,94107
3,913454,409,8/31/2015 23:10,San Jose City Hall,10,8/31/2015 23:17,San Salvador at 1st,8,68,Subscriber,95113
4,913453,789,8/31/2015 23:09,Embarcadero at Folsom,51,8/31/2015 23:22,Embarcadero at Sansome,60,487,Customer,9069


In [3]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1825 entries, 0 to 1824
Data columns (total 24 columns):
PDT                           1825 non-null object
Max TemperatureF              1821 non-null float64
Mean TemperatureF             1821 non-null float64
Min TemperatureF              1821 non-null float64
Max Dew PointF                1775 non-null float64
MeanDew PointF                1775 non-null float64
Min DewpointF                 1775 non-null float64
Max Humidity                  1775 non-null float64
 Mean Humidity                1775 non-null float64
 Min Humidity                 1775 non-null float64
 Max Sea Level PressureIn     1824 non-null float64
 Mean Sea Level PressureIn    1824 non-null float64
 Min Sea Level PressureIn     1824 non-null float64
 Max VisibilityMiles          1820 non-null float64
 Mean VisibilityMiles         1820 non-null float64
 Min VisibilityMiles          1820 non-null float64
 Max Wind SpeedMPH            1824 non-null float64
 Mean Wind

In [39]:
station_data[station_data.station_id == 41]
trip_data[trip_data['Start Terminal'] == 41]

,Trip ID,Duration,Start Date,Start Station,Start Terminal,End Date,End Station,End Terminal,Bike #,Subscriber Type,Zip Code
102,913286,1036,8/31/2015 19:17,Clay at Battery,41,8/31/2015 19:34,San Francisco Caltrain (Townsend at 4th),70,535,Subscriber,95014
137,913241,267,8/31/2015 18:54,Clay at Battery,41,8/31/2015 18:58,Howard at 2nd,63,421,Subscriber,94107
323,912981,893,8/31/2015 17:35,Clay at Battery,41,8/31/2015 17:50,2nd at Townsend,61,269,Subscriber,94111
401,912886,305,8/31/2015 17:11,Clay at Battery,41,8/31/2015 17:16,Embarcadero at Sansome,60,279,Subscriber,94133
408,912876,969,8/31/2015 17:11,Clay at Battery,41,8/31/2015 17:27,2nd at Townsend,61,449,Subscriber,94107
443,912822,762,8/31/2015 17:00,Clay at Battery,41,8/31/2015 17:13,San Francisco Caltrain (Townsend at 4th),70,589,Subscriber,94070
493,912752,662,8/31/2015 16:44,Clay at Battery,41,8/31/2015 16:55,San Francisco Caltrain (Townsend at 4th),70,334,Subscriber,94065
507,912736,964,8/31/2015 16:38,Clay at Battery,41,8/31/2015 16:54,San Francisco Caltrain (Townsend at 4th),70,544,Subscriber,94301
516,912727,1132,8/31/2015 16:37,Clay at Battery,41,8/31/2015 16:56,San Francisco Caltrain (Townsend at 4th),70,556,Subscriber,94402
531,912712,837,8/31/2015 16:30,Clay at Battery,41,8/31/2015 16:44,San Francisco Caltrain (Townsend at 4th),70,531,Subscriber,94025


In [14]:
station_data_sf = station_data[station_data.landmark == 'San Francisco']
station_data_sf

,station_id,name,lat,long,dockcount,landmark,installation
32,41,Clay at Battery,37.795001,-122.399970,15,San Francisco,8/19/2013
33,42,Davis at Jackson,37.797280,-122.398436,15,San Francisco,8/19/2013
34,45,Commercial at Montgomery,37.794231,-122.402923,15,San Francisco,8/19/2013
35,46,Washington at Kearney,37.795425,-122.404767,15,San Francisco,8/19/2013
36,47,Post at Kearney,37.788975,-122.403452,19,San Francisco,8/19/2013
37,48,Embarcadero at Vallejo,37.799953,-122.398525,15,San Francisco,8/19/2013
38,49,Spear at Folsom,37.790302,-122.390637,19,San Francisco,8/20/2013
39,50,Harry Bridges Plaza (Ferry Building),37.795392,-122.394203,23,San Francisco,8/20/2013
40,51,Embarcadero at Folsom,37.791464,-122.391034,19,San Francisco,8/20/2013
41,39,Powell Street BART,37.783871,-122.408433,19,San Francisco,8/25/2013


In [56]:
#trip_data['lat'] = [station_data.lat for x in station_data.station_id \
#                   if station_data.station_id == trip_data['Start Terminal']]
trip_data['lat'] = np.where(trip_data['Start Terminal'] == station_data['station_id'], \
                        trip_data_sf.append(station_data['lat']), trip_data['lat'])


ValueError: Series lengths must match to compare

In [5]:
#Converting 'str' objects to 'int'in col='Duration'
#list(map(int, trip_data['Duration']))

In [32]:
#Converting to datetime
trip_data['Start Date'] = pd.to_datetime(trip_data['Start Date'])
trip_data['End Date'] = pd.to_datetime(trip_data['End Date'])

In [33]:
weather_data['PDT'] = pd.to_datetime(weather_data['PDT'])

In [35]:
station_balance_data.time = pd.to_datetime(station_balance_data.time)